In [64]:
import pymysql

conn = pymysql.connect(host = "localhost", port = 3306, db="study", user = "cyhgpl", password = "1596")
try:
    conn
    print("연결성공")
except:
    print("연결실패")



연결성공


In [44]:
import pymysql
with open("./db_config", "r") as f:
    config = f.readlines()


config_dict = {}
for l in config:
    #k, v = l.split('=')
    idx = l.index('=')
    k = l[:idx]
    v = l[idx + 1:]
    config_dict[k] = v.rstrip()

conn = pymysql.connect(**config_dict)  # config_dict 언패킹

try:
    conn
    print("연결성공")
except Exception as e:
    print("연결실패", e)

def pagination(page, page_size=5):
    limit = page_size
    offset = page_size*(page-1)
    with conn.cursor() as cursor:
        #sql = 'insert into `students`(`name`, `age`, `address`) values("지영", "28", "전주");'
        #_sql = 'insert into `students`(`name`, `age`, `address`) values(%s, %s, %s);'
        __sql = 'select * from `students` where `age` > %s limit %s offset %s'
        #cursor.execute(sql)
        #cursor.execute(_sql,("최진","32","천안"))  #데이터를 하나만 넣더라도 튜플이 되게 ("최진",) 이런식으로 넣어준다.
        #cursor.executemany(_sql, [("가영","35","동해"),("민수","40","무안"),("다니엘","35","부산")])
        cursor.execute(__sql,(5, limit, offset))
        data = cursor.fetchall()        # 보통 fetchall 하고 limit과 offset으로 조절한다. 
        #data1 = cursor.fetchone()
        #data2 = cursor.fetchmany(size=5) 
    conn.commit()
    return data

pagination(2)

연결성공


((6, '성춘향', '경기', 27),
 (7, '이몽룡', '대전', 42),
 (9, '지영', '전주', 28),
 (10, '최진', '천안', 20),
 (11, '가영', '동해', 35))

In [45]:
def read_config(p:str) -> dict:
    with open(p, 'r') as f:
        config = f.readlines()
    config_dict = {}
    for l in config:
        #k, v = l.split('=')
        idx = l.index('=')
        k = l[:idx]
        v = l[idx + 1:]
        config_dict[k] = v.rstrip()

    return config_dict


read_config("../db_config copy") # 점 두개는 config


{'host': 'localhost',
 'user': 'cyhgpl',
 'password': '1596',
 'database': 'study',
 'charset': 'utf8mb4'}

In [65]:

with conn.cursor() as cur:
    #sql = 'update students set age=%s where studentid=%s'
    sql = 'delete from students where studentid ="10"'
    cur.execute(sql)
    #cur.executemany(sql, [(55, 2),(10, 5)])
conn.commit()

pagination(2)

((6, '성춘향', '경기', 27),
 (7, '이몽룡', '대전', 42),
 (9, '지영', '전주', 28),
 (11, '가영', '동해', 35),
 (12, '민수', '무안', 40))

In [62]:
conn.close()

In [73]:
with conn.cursor() as cur:
    #sql = 'create table Student(id int not null auto_increment, name varchar(255) not null, email varchar(255) not null, phone varchar(255) not null, major varchar(255) not null, primary key(id));'
    #cur.execute(sql)
    sql_insert = 'insert into Student(name, email, phone, major) values(%s, %s, %s, %s)'
    cur.executemany(sql_insert, [("a", "a@naver.com", "a1234", "math"),("b", "b@naver.com", "b1234", "Computer"),("c", "c@naver.com", "c1234", "Science"),("d", "d@naver.com", "d1234", "English")])
conn.commit()

In [82]:
table_name = "Student"
columns = ['name', 'email', 'phone', 'major']
columns = ','.join(columns)
values = ("최재", "jason@fin.a", "0109928279", "산업공")
value = ','.join(['%s']*len(values))
def insert(table_name, columns, values):
    try:
        with conn.cursor() as cur:
            sql = f'insert into {table_name}({columns}) values({value})'
            cur.execute(sql, values)
        conn.commit()
        return True
    except Exception as e:
        print(e)
        return False
insert(table_name, columns, values)

True

In [94]:
table_name = "Student"
columns = ['name', 'email', 'phone', 'major']
columns = ','.join(columns)
values = [("최재", "jason@fin.a", "0109928279", "산업공"), ("최", "jason@fin.", "010992827", "산업"), ("-", "jason@fin", "01099282", "산")]
value = ','.join(['%s']*len(values[0]))
def insert(table_name, columns, values):
    try:
        with conn.cursor() as cur:
            sql = f'insert into {table_name}({columns}) values({value})'
            cur.executemany(sql, values)
        conn.commit()
        return True
    except Exception as e:
        print(e)
        return False
insert(table_name, columns, values)

True

In [97]:
columns = "*"
table_name = "Student"
def select(columns, table_name):
    try:
        with conn.cursor() as cur:
            sql = f'select {columns} from {table_name}'
            cur.execute(sql)
            data = cur.fetchall()
            print(data)
        return True
    except Exception as e:
        return False
select(columns, table_name)

((1, 'a', 'a@naver.com', 'a1234', 'math'), (2, 'b', 'b@naver.com', 'b1234', 'Computer'), (3, 'c', 'c@naver.com', 'c1234', 'Science'), (4, 'd', 'd@naver.com', 'd1234', 'English'), (5, '최재진', 'jason@fin.ai', '01099282799', '산업공학'), (6, '최재', 'jason@fin.a', '0109928279', '산업공'), (7, '최재', 'jason@fin.a', '0109928279', '산업공'), (8, '최', 'jason@fin.', '010992827', '산업'), (9, '-', 'jason@fin', '01099282', '산'))


True

In [114]:
table_name = "Student"
email = "cyhgpl@naver.com"
id = 8
def select (table_name, email, id):
    try:
        with conn.cursor() as cur:
            sql = f'update {table_name} set email = %s where id ={id}'
            cur.execute(sql, email)
        conn.commit()
        return True
    except Exception as e:
        return False
select(table_name, email, id)

False

In [107]:
table_name = "Student"
email = "naver.com"
id = 9
with conn.cursor() as cur:
    sql = f'update {table_name} set email="cyhgpl@naver.com" where id={id}'
    cur.execute(sql)
conn.commit()

In [92]:
with open('data.csv', 'w') as f:        # with을 써야 자동으로 연결되고 닫힌다.
    f.write("오늘의 날씨")
    #txt = f.read()
    #print(txt)

with open('data.csv', 'r') as f:
    #f.write("오늘의 날씨")
    txt = f.read()
    print(txt)

오늘의 날씨


In [6]:
import pymysql

class MariaDB:
    def __init__(self, db_config):
        with open(f"./{db_config}", "r") as f:
            config = f.readlines()

        config_dict = {}
        for l in config:
            #k, v = l.split('=')
            idx = l.index('=')
            k = l[:idx]
            v = l[idx + 1:]
            config_dict[k] = v.rstrip()

        self.conn = pymysql.connect(**config_dict)  # config_dict 언패킹

        try:
            self.conn
            print("연결성공")
        except Exception as e:
            print("연결실패", e)
        
    def insert(self, table_name, columns, values):
        columns = ','.join(columns)
        value = ','.join(['%s']*len(values))
        try:
            with self.conn.cursor() as cur:
                sql = f'insert into {table_name}({columns}) values({value})'
                cur.execute(sql, values)
            self.conn.commit()
            return True
        except Exception as e:
            print(e)
            return False
    
    # 들어갈 데이터 값은 fstring으로는 안되고 , execute(sql, )여기서 sql 옆에 추가한다.
    def insert_many(self, table_name, columns, values):
        columns = ','.join(columns)
        value = ','.join(['%s']*len(values[0]))
        try:
            with self.conn.cursor() as cur:
                sql = f'insert into {table_name}({columns}) values({value})'
                cur.executemany(sql, values)
            self.conn.commit()
            return True
        except Exception as e:
            print(e)
            return False
            
    def select(self, table_name, page=None, page_size=5): #페이지를 안 주면 None이 되어서 전체가 출력된다.
        limit = page_size
        offset = page_size*(page-1)
        try:
            with self.conn.cursor() as cur:
                sql = f'select * from {table_name} limit {limit} offset {offset}'
                cur.execute(sql)
                data = cur.fetchall()  
            return data
        except Exception as e:
            return False
        
    def update(self, table_name, email, id):
        try:
            with self.conn.cursor() as cur:
                sql = f'update {table_name} set email = %s where id ={id}'
                cur.execute(sql, email)
            self.conn.commit()
            return True
        except Exception as e:
            return False
    
    def update_many(self, table_name, email, id):
        a= zip(email, id)
        b = []
        for i, j in a:
            b.append((i,j))
        try:
            with self.conn.cursor() as cur:
                sql = f'update {table_name} set email = %s where id = %s'
                cur.executemany(sql, b)
            self.conn.commit()
            return True
        except Exception as e:
            return False

    def delete(self,table_name, id):
        try:
            with self.conn.cursor() as cur:
                sql = f'delete from {table_name} where id ={id}'
                cur.execute(sql)
            self.conn.commit()
            return True
        except Exception as e:
            return False
    
    def delete_many(self,table_name,id):
        try:
            with self.conn.cursor() as cur:
                sql = f'delete from {table_name} where id =%s'
                cur.executemany(sql, id)
            self.conn.commit()
            return True
        except Exception as e:
            return False

md = MariaDB("db_config")
#md.insert_many("Student",["name", "email", "phone", "major"],  [("daum", "daum@daum.net", "0109928279", "Internet_of_Thing"), ("dau", "daum@daum.ne", "010992827", "Internet_of_Thin"),("da", "daum@daum.n", "01099282", "Internet_of_Thi")])
#md.update_many("student", ["update2@naver.com", "update2@naver.com", "update3@naver.com"] , [1,2,3])
md.delete_many("student",[5,6])
md.select("student", 1)

연결성공


((1, 'a', 'update2@naver.com', 'a1234', 'math'),
 (2, 'b', 'update2@naver.com', 'b1234', 'Computer'),
 (4, 'd', 'd@naver.com', 'd1234', 'English'),
 (7, '최재', 'update@naver.com', '0109928279', '산업공'),
 (8, '최', 'cyhgpl@naver.com', '010992827', '산업'))

In [198]:
a= zip(["update2@naver.com", "update2@naver.com", "update3@naver.com"] , [10,11,12])
b = []
for i, j in a:
    b.append((j,i))

print(b)



[(10, 'update2@naver.com'), (11, 'update2@naver.com'), (12, 'update3@naver.com')]
